In [7]:
pip install peft trl;

In [3]:
pip install wandb;

   ---------------------------------------- 0.0/22.5 MB ? eta -:--:--
   - -------------------------------------- 1.0/22.5 MB 7.1 MB/s eta 0:00:04
   ---- ----------------------------------- 2.4/22.5 MB 6.7 MB/s eta 0:00:04
   ------ --------------------------------- 3.4/22.5 MB 6.7 MB/s eta 0:00:03
   ------ --------------------------------- 3.9/22.5 MB 5.5 MB/s eta 0:00:04
   ------- -------------------------------- 4.5/22.5 MB 4.9 MB/s eta 0:00:04
   -------- ------------------------------- 5.0/22.5 MB 4.4 MB/s eta 0:00:05
   --------- ------------------------------ 5.5/22.5 MB 4.1 MB/s eta 0:00:05
   ---------- ----------------------------- 6.0/22.5 MB 3.8 MB/s eta 0:00:05
   ----------- ---------------------------- 6.6/22.5 MB 3.6 MB/s eta 0:00:05
   ------------ --------------------------- 7.1/22.5 MB 3.5 MB/s eta 0:00:05
   ------------- -------------------------- 7.6/22.5 MB 3.4 MB/s eta 0:00:05
   -------------- ------------------------- 8.1/22.5 MB 3.3 MB/s eta 0:00:05
   ---

In [2]:
from datasets import load_dataset, Dataset
import pandas as pd
df = pd.read_json('nl_sparql_pairs_500.json')
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['prompt', 'messages', 'prompt_id'],
    num_rows: 500
})

In [2]:
import wandb
wandb.init()


In [ ]:
import sys
import logging
import datasets
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import torch
import transformers
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig

logger = logging.getLogger(__name__)

# Training config
training_config = {
    "bf16": False,
    "do_eval": False,
    "learning_rate": 5.0e-06,
    "log_level": "info",
    "logging_steps": 20,
    "logging_strategy": "steps",
    "lr_scheduler_type": "cosine",
    "num_train_epochs": 3,
    "max_steps": -1,
    "output_dir": "./checkpoint_dir",
    "overwrite_output_dir": True,
    "per_device_eval_batch_size": 2,
    "per_device_train_batch_size": 2,
    "remove_unused_columns": True,
    "save_steps": 100,
    "save_total_limit": 1,
    "seed": 0,
    "gradient_checkpointing": True,
    "gradient_checkpointing_kwargs": {"use_reentrant": False},
    "gradient_accumulation_steps": 4,
    "warmup_ratio": 0.2,
}

# LoRA config
peft_config = {
    "r": 8,
    "lora_alpha": 16,
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    "modules_to_save": None,
}

train_conf = TrainingArguments(**training_config)
peft_conf = LoraConfig(**peft_config)

# Logging setup
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
log_level = train_conf.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

logger.warning(
    f"Process rank: {train_conf.local_rank}, device: {train_conf.device}, n_gpu: {train_conf.n_gpu}"
    + f" distributed training: {bool(train_conf.local_rank != -1)}, 16-bits training: {train_conf.fp16}"
)
logger.info(f"Training/evaluation parameters {train_conf}")
logger.info(f"PEFT parameters {peft_conf}")

# Model loading
checkpoint_path = "microsoft/Phi-3-mini-4k-instruct"
model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
    device_map=None
)
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, **model_kwargs)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer.model_max_length = 2048
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = "right"

# Data processing
def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    example["text"] = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False
    )
    return example

# Placeholder dataset (replace with your SPARQL dataset)
train_dataset = dataset
test_dataset = dataset
column_names = list(train_dataset.features)

processed_train_dataset = train_dataset.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
    num_proc=10,
    remove_columns=column_names,
    desc="Applying chat template to train_sft",
)
processed_test_dataset = test_dataset.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
    num_proc=10,
    remove_columns=column_names,
    desc="Applying chat template to test_sft",
)

# Training
trainer = SFTTrainer(
    model=model,
    args=train_conf,
    peft_config=peft_conf,
    train_dataset=processed_train_dataset,
    eval_dataset=processed_test_dataset,
)
train_result = trainer.train()
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

# Evaluation
tokenizer.padding_side = "left"
metrics = trainer.evaluate()
metrics["eval_samples"] = len(processed_test_dataset)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

# Save model
trainer.save_model(train_conf.output_dir)

[INFO|training_args.py:2135] 2025-06-25 09:15:10,900 >> PyTorch: setting up devices
[INFO|training_args.py:1812] 2025-06-25 09:15:10,903 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


2025-06-25 09:15:10 - WARNING - __main__ - Process rank: 0, device: cpu, n_gpu: 0 distributed training: True, 16-bits training: False
2025-06-25 09:15:10 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_pr

[INFO|configuration_utils.py:698] 2025-06-25 09:15:12,487 >> loading configuration file config.json from cache at C:\Users\Kanish Prabakaran\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct\snapshots\0a67737cc96d2554230f90338b163bc6380a2a85\config.json
[INFO|configuration_utils.py:698] 2025-06-25 09:15:12,806 >> loading configuration file config.json from cache at C:\Users\Kanish Prabakaran\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct\snapshots\0a67737cc96d2554230f90338b163bc6380a2a85\config.json
[INFO|configuration_utils.py:770] 2025-06-25 09:15:12,810 >> Model config Phi3Config {
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hi

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:  38%|###8      | 3.09G/8.07G [00:00<?, ?B/s]